In [ ]:
import cv2
import torch
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive')

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-3-28 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
#capture = cv2.VideoCapture("lostark sample.mp4")
capture = cv2.VideoCapture("LOSTARK 2022-03-26 16-56-16-446.mp4")

capture_w = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
capture_h = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
capture_total_frame = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

fps = capture.get(cv2.CAP_PROP_FPS)

print("{0} x {1}".format(capture_w, capture_h))
print(capture_total_frame)
print(fps)

1920 x 1080
10919
29.86869972359804


In [ ]:
import ipywidgets as widgets
slider = widgets.IntSlider(value=5, max=capture_total_frame)
display(slider)

IntSlider(value=5, max=10919)

In [ ]:
capture.set(cv2.CAP_PROP_POS_FRAMES, slider.value)
_, bg = capture.read()
cv2_imshow(bg)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
list_skill_alpha = ["Q", "W", "E", "R",
                    "A", "S", "D", "F"]

list_skill = [(682, 978, 45, 45), (729, 978, 45, 45), (776, 978, 45, 45), (823, 815, 45, 45),
              (704, 1025, 45, 45), (751, 1025, 45, 45), (798, 1025, 45, 45), (845, 1025, 45, 45)]

# 확인하고자 하는 스킬 입력
input_skill = input("확인하려는 스킬은? ")

sk_idx = 0
while input_skill != list_skill_alpha[sk_idx]:
    sk_idx = sk_idx+1

roi_x1 = int(list_skill[sk_idx][0])
roi_x2 = int(list_skill[sk_idx][0]+list_skill[sk_idx][2])

roi_y1 = int(list_skill[sk_idx][1])
roi_y2 = int(list_skill[sk_idx][1]+list_skill[sk_idx][3])

확인하려는 스킬은? S
751
796
1025
1070


In [ ]:
# 배경 설정
#capture.set(cv2.CAP_PROP_POS_FRAMES, int(capture_total_frame/10))
capture.set(cv2.CAP_PROP_POS_FRAMES, 5)
_, bg = capture.read()
img_bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)
roi_bg = img_bg[roi_y1:roi_y2, roi_x1:roi_x2] # (y, x)
cut_bg = img_bg[0:25, 110:333]

# 프레임 초기화
capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

True

In [ ]:
skill_use_frame_mean_list = [0]
cutscene_mean_list = [0]
skill_use_frame_idx_list = []

for i in range(1, int(capture_total_frame)):
    run, frame = capture.read()
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = img[roi_y1:roi_y2, roi_x1:roi_x2] # (y, x)
    cut = img[0:25, 110:333]

    skill_use = cv2.absdiff(roi, roi_bg)
    mean_value_skill_use = float(cv2.mean(skill_use)[0])

    cutscene_diff = cv2.absdiff(cut, cut_bg)
    cutscene_chk = float(cv2.mean(cutscene_diff)[0])

    skill_use_frame_mean_list.append(mean_value_skill_use)
    skill_used = abs(skill_use_frame_mean_list[i-1]-skill_use_frame_mean_list[i])
    
    cutscene_mean_list.append(cutscene_chk)
    cutscene = abs(cutscene_mean_list[i-1]-cutscene_mean_list[i])
    
    if skill_used > 70 and cutscene < 30:
      skill_use_frame_idx_list.append(int(capture.get(cv2.CAP_PROP_POS_FRAMES)))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
headcnt = 0
model.conf = 0.9
for i in range(len(skill_use_frame_idx_list)):
    capture.set(cv2.CAP_PROP_POS_FRAMES, int(skill_use_frame_idx_list[i])+10)
    while True:
      run2, frame2 = capture.read()

      if i < len(skill_use_frame_idx_list)-1:
        if skill_use_frame_idx_list[i+1] - skill_use_frame_idx_list[i] > 60:
          magic_num = 60
        else:
          magic_num = int((skill_use_frame_idx_list[i+1] - skill_use_frame_idx_list[i])/2)

      if capture.get(cv2.CAP_PROP_POS_FRAMES) >= int(skill_use_frame_idx_list[i]+magic_num):
        break
      if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture_total_frame:
        break
      #print(capture.get(cv2.CAP_PROP_POS_FRAMES))

      img2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
      area1 = img2[220:860, 640:1280]
      area2 = img2[220:860, 320:960]
      area3 = img2[220:860, 960:1600]
      area4 = img2[60:700, 640:1280]

      detecting1 = model(area1)
      if len(detecting1.pandas().xyxy[0])>0:
        headcnt = headcnt+1
        detecting1.save()
        print(capture.get(cv2.CAP_PROP_POS_FRAMES))
        break
      else:
        detecting4 = model(area4)
        if len(detecting4.pandas().xyxy[0])>0:
          headcnt = headcnt+1
          detecting4.save()
          print(capture.get(cv2.CAP_PROP_POS_FRAMES))
          break
        else:
          detecting2 = model(area2)
          if len(detecting2.pandas().xyxy[0])>0:
            headcnt = headcnt+1
            detecting2.save()
            print(capture.get(cv2.CAP_PROP_POS_FRAMES))
            break
          else:
            detecting3 = model(area3)
            if len(detecting3.pandas().xyxy[0])>0:
              headcnt = headcnt+1
              detecting3.save()
              print(capture.get(cv2.CAP_PROP_POS_FRAMES))
              break

print(headcnt)
print(len(skill_use_frame_idx_list))
print(headcnt/len(skill_use_frame_idx_list))

Saved 1 image to runs/detect/exp3


1286.0


Saved 1 image to runs/detect/exp4


1981.0


Saved 1 image to runs/detect/exp5


4252.0


Saved 1 image to runs/detect/exp6


8105.0
4
6
0.6666666666666666
